In [4]:
import requests
import time
import os,sys
import pickle

In [5]:
passed_obj = []
id_genres_obj = []
def get_id_genres(song_id,ite,stop):
    global passed_obj,id_genres_obj
    if ite == 0:
        passed_obj.append(song_id)
    if ite >= stop:
        return
    res = requests.get("https://mp3.zing.vn/xhr/media/get-info?type=audio&id="+song_id)
    try:
        genres = dict(res.json())['data']['genres']
    except:
        genres = []
        print("res")
    res_rev = requests.get("http://mp3.zing.vn/xhr/recommend?type=audio&id="+song_id)
    rev = dict(res_rev.json())['data']['items']
    id_genres_obj.append((song_id,genres))
    print(song_id,[ele['id'] for ele in rev])
    for ele in rev:
        next_id = ele['id']
        if next_id in passed_obj:
            continue
        passed_obj.append(next_id)
        get_id_genres(next_id,ite + 1,stop)
        
def get_info_from_name(name,stop):
    global passed_obj,id_genres_obj
    passed_obj = []
    id_genres_obj = []
    res_by_name = requests.get("http://ac.mp3.zing.vn/complete?type=song&num=500&query="+name)
    t = (res_by_name.json())['data']
    if t:  
        t = t[0]
        song_id = t['song'][0]['id']
        get_id_genres(song_id,0,stop)
    else:
        return []
    return id_genres_obj

def get_info_from_list_of_name(names,stop):
    re = []
    for ele in names:
        re.extend(get_info_from_name(ele,stop))
    return re

def download_from_id(idx,path):
    r = requests.get("http://api.mp3.zing.vn/api/streaming/audio/"+idx+"/320",allow_redirects=True)
    with open(path +".wav","wb+") as f:
        f.write(r.content)
        
def get_songs_data(names,org_path,stop):
    re = get_info_from_list_of_name(names,stop)
    print(re)
    for idx,gen in re:
        tmp = os.path.join(org_path,idx)
        os.mkdir(tmp)
        download_from_id(idx,os.path.join(tmp,idx))
        with open(os.path.join(tmp,"genre"),'wb+') as f:
            pickle.dump(gen,f)


In [8]:
#Dien ten viet hoa cua vai bai hat - dinh dang: moi tu trong ten viet hoa chu dau, co' da^u'. vd: Tinh Ve Noi Dau
names = ["Love Story", "Perfect", "Bad Liar"]
#Do sau tim kiem --> cho vua phai thoi
stop = 3
#Dia chi thu muc luu du lieu
org_path = './'
data_path = os.path.join(org_path,"music_data")
if not os.path.exists(data_path):
    os.mkdir(data_path)
    
get_songs_data(names,data_path,stop)

ZW7889UB ['ZW7IAEZA', 'ZW77FC60', 'ZW78DIOA', 'ZW78IFUB', 'ZW77F7C8', 'ZW7809ED', 'ZW78WBZ9', 'ZW78I6DI', 'ZWAEUIOW', 'ZWAEUIW8']
ZW7IAEZA ['ZW77F7C8', 'ZW7809ED', 'ZW7ODFI0', 'ZW78I6DI', 'ZW78IFUB', 'ZW77FC60', 'ZWB0B8D0', 'ZW78IZA9', 'ZWB0B6UE', 'ZWAEUIOW']
ZW77F7C8 ['ZW78I6DI', 'ZW77FC60', 'ZWAEUIOW', 'ZW7IAEZA', 'ZW7ODFI0', 'ZW77F8UZ', 'ZWB0B8D0', 'ZW78IZA9', 'ZW7809ED', 'ZW78IFUB']
ZWB0B6UE ['ZW78IZA9', 'ZW7ODFI0', 'ZW6F98CB', 'ZW78I6DI', 'ZWBIZI7W', 'ZW77FC60', 'ZW7IAEZA', 'ZWAEUIOW', 'ZWB0B8D0', 'ZW77F7C8']
ZW78DIOA ['ZW78I6DI', 'ZW77FC60', 'ZW7UBUCC', 'ZWAEUIWB', 'ZW7809ED', 'ZW77FC6W', 'ZWB078FC', 'ZWAEUIOW', 'ZW77F8U0', 'ZW78IF9U']
ZW7UBUCC ['ZW77F8U0', 'ZW78WBZ9', 'ZW78O8FA', 'ZWAEUIWB', 'ZW78DIOA', 'ZW78IF97', 'ZW7809ED', 'ZW78I6DI', 'ZWAEZUU7', 'ZW77FC6W']
ZWB078FC ['ZW78DIOA', 'ZWB00AW7', 'ZWAEUIW8', 'ZWAEUIOW', 'ZW7809ED', 'ZWBIZI7W', 'ZW78WBZ9', 'ZWBWB7DU', 'ZW78I6DI', 'ZWB0U60U']
ZW78IF9U ['ZW6OAO9E', 'ZW78B0CU', 'ZW78IF97', 'ZW78O8FA', 'ZW78DIOA', 'ZW7UBUCC', 'ZWAEZUU